### 五年人口迁徙图

In [1]:
import re
import pandas as pd
import requests as rq
from iclientpy import viz

使用pandas读取数据，但是列错乱，修正后生成新的dataframe

In [2]:
sdf = pd.read_html(rq.get('http://www.stats.gov.cn/tjsj/pcsj/rkpc/6rp/html/B0708.htm', headers={'User-agent': 'Mozilla/5.0'}).content)[0]
df=sdf.iloc[8:39,[0]+list(range(3,34))]
df.columns=['现住地']+sdf.iloc[5,1:32].tolist()

从源数据获取各地方迁徙总人口的dataframe，然后提取迁徙人口最多的五个省份

In [3]:
tdf=sdf.iloc[7:8,3:34]
tdf.columns=sdf.iloc[5,1:32].tolist()
tdf.loc[7]=pd.to_numeric(tdf.loc[7])
ttdf=tdf.T
ttdf.columns=['value']
sttdf=ttdf.sort_values(by='value',ascending=False)
top5_name=sttdf.iloc[0:5].index.tolist()

构建geolines所需要的数据格式

In [4]:
def get_data(dataframe):
    alldata=[]
    for index,series in dataframe.iteritems():
        if index !=  "现住地":
            single_data=[]
            for ind,value in series.iteritems():
                if  index != re.sub('\s','',dataframe['现住地'][ind]):
                   single_data.append([{'name': index},{'name': re.sub('\s','',dataframe['现住地'][ind]),'value':value}]) 
            alldata.append(single_data)
    return alldata

In [5]:
data=get_data(df)

根据流出人口数量计算示例动画的大小

In [6]:
def compute_symbolsize(data):
    max_sub=sttdf.iloc[0,0]-sttdf.iloc[-1,0]
    val_sub=sttdf.loc[data[0][0]["name"]][0]-sttdf.iloc[-1,0]
    nor_div = val_sub/max_sub
    return int(10*nor_div+5)

生成迁徙图

In [7]:
chart=viz.geolines(data=data,address_key='name',value_key='value',names=df.columns.tolist()[1:],symbol_size=compute_symbolsize,symbol='arrow',selected_mode='multiple',selected_legend=top5_name)

展示迁徙图

In [8]:
chart

TypeError: compute_symbolsize() takes 1 positional argument but 2 were given

GeoLines()